In [2]:
#各ライブラリのimport
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
#データ読み込みから標準化まで(train,test結合版)
#ダミー変数も標準化
#欠損値補完と標準化の間にエンコーディングの操作を入れるとエラーが出てしまったため改善しよう
#データ読み込み
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')
df_train_test = pd.concat([df_train , df_test] , axis =0)
#print(df_train_test[df_train_test['horsepower'] == "?"])
df_train_test['horsepower'] = df_train_test['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出
#欠損値補完
#amc concord dl
df_train.iloc[24 , 4] = df_train_test[df_train_test['displacement'] == 151]['horsepower'].mean()
#renault 18
df_train.iloc[113 , 4] = df_train_test[(98 <= df_train_test['displacement']) & (df_train_test['displacement']<= 102)]['horsepower'].mean()
#renault lecar deluxe
df_train.iloc[145 , 4] = df_train_test[df_train_test['displacement'] == 85]['horsepower'].mean()
#ford pinto
df_train.iloc[175 , 4] = df_train_test[df_train_test['car name'] == 'ford pinto']['horsepower'].mean()
#ford maverick
df_test.loc[70 , 'horsepower'] = df_train_test[df_train_test['car name'] == "ford maverick"]['horsepower'].mean()
#ford mustang cobra
df_test.loc[112 , 'horsepower'] = df_train_test[df_train_test['displacement'] == 140]['horsepower'].mean()
#補完確認
#df_train_test[df_train_test['horsepower'].isnull()]
#ラベルエンコーディング(train,test結合版)
#メーカーカラム作成
df_split = df_train['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train_test['manufacturers name'] = df_split.iloc[:, 0]
df_train_test['manufacturers name'] = df_train_test['manufacturers name'].replace({'chevroelt':'chevrolet' , 'toyouta':'toyota'})#カラム名の修正

#エンコーディング
from sklearn.preprocessing import OneHotEncoder
ohencoder = OneHotEncoder(sparse=False , dtype=int)
#メーカーごとにone-hot
df_manufacturers_number= pd.DataFrame(ohencoder.fit_transform(df_train_test['manufacturers name'].values.reshape(-1,1)) , columns = df_train_test['manufacturers name'].unique()).astype(int)
#国ごとにone-hot
df_origin_number= pd.DataFrame(ohencoder.fit_transform(df_train_test['origin'].values.reshape(-1,1)) , columns = ['America' , 'Europe' , 'Japan']).astype(int)
#ダミー変数に変換したdfの結合
df_number = pd.merge(df_manufacturers_number , df_origin_number , how = 'outer' , right_index = True , left_index=True)
df_number_id = pd.merge(df_number , df_train_test, how = 'outer' , right_index = True , left_index=True)
#df_numberとdf_train_testの結合
#右側のデータフレームのインデックスをキーとして、左側のデータフレームを結合する(right_index=True)
#右側のデータフレームのインデックスをキーとして、左側のデータフレームを結合する(right_index=True)
#left_indexとright_indexの両方をTrueにしないとエラーになってしまった
df_train = pd.merge(df_train , df_number_id , how = 'outer'  , on = 'id')
df_train = df_train.dropna(how = 'any')
print(df_train.columns)
df_test = pd.merge(df_test , df_number_id , how = 'outer' , on = 'id')
print(df_test)
df_test = df_test.dropna(how = 'any')
print(df_test)
df_test.head()
#標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_drop = df_train.drop(['id' , 'car name' , 'origin'] , axis = 1)
df_test_drop = df_test.drop(['id' , 'car name' , 'origin'] , axis = 1)
#df_testにmpgカラムがないからdf_trainのfitの計算結果を使えない
#別々にfitして標準化した
scaler.fit(df_train_drop)
df_train_std= pd.DataFrame(scaler.transform(df_train_drop), columns = df_train_drop.columns)
scaler.fit(df_test_drop)
df_test_std = pd.DataFrame(scaler.transform(df_test_drop), columns = df_test_drop.columns)

#目的変数の分離
features_train = df_train_std.drop(['weight' , 'acceleration' , 'mpg' ] , axis = 1)
target_train = df_train_std['mpg']
features_test = df_test_std.drop(['weight' , 'acceleration'] , axis = 1)
#学習
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100 , random_state = 0)
model.fit(features_train , target_train)
#予測
df_pred = pd.DataFrame(model.predict(features_test))
df_test = pd.read_csv('../test.tsv' , sep = '\t')
df_pred = pd.merge(df_test['id'].astype(int) , df_pred , how = 'outer' , right_index = True , left_index=True)
df_pred
#評価
df_pred.to_csv('../RF_sample_submit.csv' , header = False , index = False)


c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Index(['id', 'mpg_x', 'cylinders_x', 'displacement_x', 'horsepower_x',
       'weight_x', 'acceleration_x', 'model year_x', 'origin_x', 'car name_x',
       'dodge', 'vw', 'toyota', 'pontiac', 'chevrolet', 'oldsmobile', 'bmw',
       'mercedes-benz', 'datsun', 'amc', 'renault', 'peugeot', 'ford',
       'mercury', 'subaru', 'honda', 'volkswagen', 'saab', 'mazda', 'plymouth',
       'opel', 'chevy', 'capri', 'fiat', 'hi', 'audi', 'buick', 'vokswagen',
       'volvo', 'triumph', 'America', 'Europe', 'Japan', 'mpg_y',
       'cylinders_y', 'displacement_y', 'horsepower_y', 'weight_y',
       'acceleration_y', 'model year_y', 'origin_y', 'car name_y',
       'manufacturers name'],
      dtype='object')
      id  cylinders_x  displacement_x horsepower_x  weight_x  acceleration_x  \
0    1.0          6.0           145.0        76.00    3160.0            19.6   
1    2.0          6.0           250.0        98.00    3525.0            19.0   
2    4.0          4.0           119.0        92.00  

KeyError: "['car name', 'origin'] not found in axis"

In [8]:
#データ読み込みから標準化まで(train,test結合版)

#欠損値補完と標準化の間にエンコーディングの操作を入れるとエラーが出てしまったため改善しよう
#データ読み込み
df_train = pd.read_csv('データ/train.tsv' , sep = '\t')
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')
df_train_test = pd.concat([df_train , df_test] , axis =0)
#print(df_train_test[df_train_test['horsepower'] == "?"])
df_train_test['horsepower'] = df_train_test['horsepower'].replace({'?':np.nan}).astype(float)
df_train[df_train['horsepower'].isnull()]#欠損値を含むカラムの抽出
#欠損値補完
#amc concord dl
df_train.iloc[24 , 4] = df_train_test[df_train_test['displacement'] == 151]['horsepower'].mean()
#renault 18
df_train.iloc[113 , 4] = df_train_test[(98 <= df_train_test['displacement']) & (df_train_test['displacement']<= 102)]['horsepower'].mean()
#renault lecar deluxe
df_train.iloc[145 , 4] = df_train_test[df_train_test['displacement'] == 85]['horsepower'].mean()
#ford pinto
df_train.iloc[175 , 4] = df_train_test[df_train_test['car name'] == 'ford pinto']['horsepower'].mean()
#ford maverick
df_test.loc[70 , 'horsepower'] = df_train_test[df_train_test['car name'] == "ford maverick"]['horsepower'].mean()
#ford mustang cobra
df_test.loc[112 , 'horsepower'] = df_train_test[df_train_test['displacement'] == 140]['horsepower'].mean()
#補完確認
#df_train_test[df_train_test['horsepower'].isnull()]
#ラベルエンコーディング(train,test結合版)
#メーカーカラム作成
df_split = df_train['car name'].str.split(expand = True)#str.split('')で文字列を''で分割(入力しないと空白で分割される)
df_train_test['manufacturers name'] = df_split.iloc[:, 0]
df_train_test['manufacturers name'] = df_train_test['manufacturers name'].replace({'chevroelt':'chevrolet' , 'toyouta':'toyota'})#カラム名の修正

#エンコーディング
from sklearn.preprocessing import OneHotEncoder
ohencoder = OneHotEncoder(sparse=False , dtype=int)
#メーカーごとにone-hot
df_manufacturers_number= pd.DataFrame(ohencoder.fit_transform(df_train_test['manufacturers name'].values.reshape(-1,1)) , columns = df_train_test['manufacturers name'].unique()).astype(int)
#国ごとにone-hot
df_origin_number= pd.DataFrame(ohencoder.fit_transform(df_train_test['origin'].values.reshape(-1,1)) , columns = ['America' , 'Europe' , 'Japan']).astype(int)
#ダミー変数に変換したdfの結合
df_number = pd.merge(df_manufacturers_number , df_origin_number , how = 'outer' , right_index = True , left_index=True)

#標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_drop = df_train.drop([ 'car name' , 'origin' , 'mpg'] , axis = 1)
df_test_drop = df_test.drop(['car name' , 'origin'] , axis = 1)
#df_testにmpgカラムがないからdf_trainのfitの計算結果を使えない
#別々にfitして標準化した
scaler.fit(df_train_drop)
df_train_std= pd.DataFrame(scaler.transform(df_train_drop), columns = df_train_drop.columns)
#scaler.fit(df_test_drop)
df_test_std = pd.DataFrame(scaler.transform(df_test_drop), columns = df_test_drop.columns)

#df_numberとdf_train_testの結合
#右側のデータフレームのインデックスをキーとして、左側のデータフレームを結合する(right_index=True)
#右側のデータフレームのインデックスをキーとして、左側のデータフレームを結合する(right_index=True)
#left_indexとright_indexの両方をTrueにしないとエラーになってしまった
df_train = pd.merge(df_train_std , df_number , how = 'outer' , right_index = True , left_index=True)
#print(df_train)
df_train = df_train.dropna(how = 'any')
#print(df_train)
df_test = pd.merge(df_test_std , df_number , how = 'outer' , right_index = True , left_index=True)
print(df_test)
df_test = df_test.dropna(how = 'any')
print(df_test)
df_test.head()

           id  cylinders  displacement  horsepower    weight  acceleration  \
0   -1.760278   0.428863     -0.390325   -0.703290  0.337727      1.466468   
1   -1.751440   0.428863      0.679434   -0.079935  0.784098      1.243841   
2   -1.733764  -0.790333     -0.655218   -0.249941 -0.550124     -0.240340   
3   -1.724926   0.428863      0.760940    0.260076  0.095586     -0.796909   
4   -1.716088  -0.790333     -0.879358   -0.363278 -0.958585      0.316228   
..        ...        ...           ...         ...       ...           ...   
393       NaN        NaN           NaN         NaN       NaN           NaN   
394       NaN        NaN           NaN         NaN       NaN           NaN   
395       NaN        NaN           NaN         NaN       NaN           NaN   
396       NaN        NaN           NaN         NaN       NaN           NaN   
397       NaN        NaN           NaN         NaN       NaN           NaN   

     model year  dodge  vw  toyota  ...  fiat  hi  audi  buick 

c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\shouh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,id,cylinders,displacement,horsepower,weight,acceleration,model year,dodge,vw,toyota,...,fiat,hi,audi,buick,vokswagen,volvo,triumph,America,Europe,Japan
0,-1.760278,0.428863,-0.390325,-0.703290,0.337727,1.466468,1.274377,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,-1.751440,0.428863,0.679434,-0.079935,0.784098,1.243841,0.219903,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,-1.733764,-0.790333,-0.655218,-0.249941,-0.550124,-0.240340,1.010758,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,-1.724926,0.428863,0.760940,0.260076,0.095586,-0.796909,-1.361808,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,-1.716088,-0.790333,-0.879358,-0.363278,-0.958585,0.316228,-1.098190,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [10]:
#目的変数の分離
features_train = df_train.drop(['weight' , 'acceleration' ] , axis = 1)
df_train = pd.read_csv('データ/train.tsv' , sep = '\t)
target_train = df_train['mpg']
features_test = df_test.drop(['weight' , 'acceleration'] , axis = 1)
#学習
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100 , random_state = 0)
model.fit(features_train , target_train)
#予測
df_pred = pd.DataFrame(model.predict(features_test))
df_test = pd.read_csv('データ/test.tsv' , sep = '\t')
df_pred = pd.merge(df_test['id'].astype(int) , df_pred , how = 'outer' , right_index = True , left_index=True)
df_pred
#評価
df_pred.to_csv('../RF_sample_submit.csv' , header = False , index = False)

KeyError: 'mpg'